In [1]:
#Loading text data from transcript
from langchain_community.document_loaders import TextLoader
loader= TextLoader('transcript.txt')
text_document =loader.load()
text_document

[Document(metadata={'source': 'transcript.txt'}, page_content="I wake up in the morning and I want to reach for my phone, but I know that even if I were to crank up the brightness on\nthat phone screen, it's not bright enough to trigger that cortisol spike, and for me to be at my most alert and focused\nthroughout the day and to optimize my sleep at night. So what I do is I get out of bed and I go outside, and if it's a\nbright, clear day, and the sun is low in the sky, or the sun is starting to get overhead, what we call low solar angle,\nthen I know I'm getting outside at the right time. If there's cloud cover and I can't see the sun, I also know I'm doing\na good thing because it turns out especially on cloudy days, you want to get outside and get as much light energy or \nphotons in your eyes. But let's say it's a very clear day and I can see where the sun is, I do not need to stare \ndirectly into the sun. If it's very low in the sky, I might do that because it's not going to be v

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY')



In [3]:
#Chunking
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter= RecursiveCharacterTextSplitter(
    chunk_size= 500,
    chunk_overlap=0
)
documents= text_splitter.split_documents(text_document)

documents

[Document(metadata={'source': 'transcript.txt'}, page_content="I wake up in the morning and I want to reach for my phone, but I know that even if I were to crank up the brightness on\nthat phone screen, it's not bright enough to trigger that cortisol spike, and for me to be at my most alert and focused\nthroughout the day and to optimize my sleep at night. So what I do is I get out of bed and I go outside, and if it's a\nbright, clear day, and the sun is low in the sky, or the sun is starting to get overhead, what we call low solar angle,"),
 Document(metadata={'source': 'transcript.txt'}, page_content="then I know I'm getting outside at the right time. If there's cloud cover and I can't see the sun, I also know I'm doing\na good thing because it turns out especially on cloudy days, you want to get outside and get as much light energy or \nphotons in your eyes. But let's say it's a very clear day and I can see where the sun is, I do not need to stare \ndirectly into the sun. If it's ve

In [4]:
#Embedding and vector DB storage
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma

# Initialize an embedding model and vector store
embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
from langchain_community.vectorstores import FAISS
db= FAISS.from_documents(documents, embedding)

c:\Users\ASUS\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
#querying/sanity test
query= 'What is Huberman saying about morning sunlight?'
result= db.similarity_search(query)
result

[Document(metadata={'source': 'transcript.txt'}, page_content="that's so bright that it's painful to look at 'cause you can damage your eyes. But for this morning sunlight viewing, it's \nbest to not wear sunglasses. That's right, to not wear sunglasses at least for this morning sunlight viewing. It is \nabsolutely fine to wear eyeglasses or contact lenses, so-called corrective lenses. In fact, those will serve you well \nin this practice or this tool because they will focus the light onto your neural retina and onto those melanopsin"),
 Document(metadata={'source': 'transcript.txt'}, page_content="intrinsically photosensitive ganglion cells. If your eyeglasses or contact lenses have UV protection, that's okay. \nThere's so many different wavelengths of light coming from the sun and they are bright enough that they will trigger \nthe mechanisms that you want triggered at this early time of day. So try and get outside, ideally within the first five \nminutes of waking or maybe it's 15 m

In [8]:
#LLM for RAG pipeline
from langchain_ollama.llms import OllamaLLM
llm = OllamaLLM(model= "llama3")
llm

OllamaLLM(model='llama3')

In [9]:
#Designing chatprompt template
from langchain_core.prompts import ChatPromptTemplate
prompt= ChatPromptTemplate.from_template("""
Answer the question asked based only on the provided context.
Think in-depth before giving answer, but do not try to use any external sources
of information.
<context>
{context}
</context>
Question: {input}

""")

In [10]:
#Chain introduction
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain= create_stuff_documents_chain(llm, prompt)


In [11]:
retriever= db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000209E77F8DD0>)

In [13]:
#Retrieval chain

from langchain.chains import create_retrieval_chain
retrieval_chain= create_retrieval_chain(retriever, document_chain)
user_query= input("What do you want to ask?")
response= retrieval_chain.invoke({"input": user_query})
response['answer']